In [ ]:
s = 1e-5

import pandas as pd
from sklearn import metrics
import tensorflow as tf
################################ MSE ################################
def MSE(y_true, y_pred):
    return tf.reduce_mean(tf.math.square(y_true-y_pred))

################################ BCE ################################
def BCE(y_true, y_pred):
    return -tf.reduce_mean(y_true*tf.math.log(y_pred+s)+(1-y_true)*tf.math.log(1-y_pred+s))

################################ WBCE ################################
def WBCE(y_true, y_pred):
    N = batch    # batch_size
    y1 = tf.reduce_sum(y_true)
    y0 = N-y1
    w1 = y0/N #N/y1
    w0 = y1/N #N/y0
    return -tf.reduce_mean(w1*y_true*tf.math.log(y_pred+s)+w0*(1-y_true)*tf.math.log(1-y_pred+s))

################################ TN/FP/FN/TP ################################
def confusion_matrix(y_true, y_pred):
    N = batch    # batch_size
    y1 = tf.reduce_sum(y_true)
    y0 = N-y1
    TN = N-tf.reduce_sum(y_true)-tf.reduce_sum(y_pred)+tf.reduce_sum(y_true*y_pred)
    FP = tf.reduce_sum(y_pred)-tf.reduce_sum(y_true*y_pred)
    FN = tf.reduce_sum(y_true)-tf.reduce_sum(y_true*y_pred)
    TP = tf.reduce_sum(y_true*y_pred)
    return N, y1, y0, TN, FP, FN, TP
    
################################ SPL ################################
def splitter(y_pred):
    return (0.5)**2-(y_pred-0.5)**2


# =================================== Fbeta (Type0, 1, 2, 3) =================================== #
################################ Pure_Fbeta ################################
def Pure_Fbeta(y_true, y_pred):
    b = 1 
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return 1-F_beta

################################ Any_Fbeta ################################
def Any_Fbeta(y_true, y_pred):
    b = 1 
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return 1-F_beta

################################ WBCEFL ################################
def WBCEFL(y_true, y_pred):
    b = 1
    WBCEloss = WBCE(y_true, y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return (1-r)*WBCEloss+(r)*(1-F_beta)

################################ SPLFL ################################
def SPLFL(y_true, y_pred):
    b = 1
    SPL = splitter(y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return (1-w)*SPL+(w)*(1-F_beta)

# =================================== Gmean (Type0, 1, 2, 3) =================================== #
################################ Pure_Gmean ################################
def Pure_Gmean(y_true, y_pred):
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return 1-sur_gmean

################################ Any_Gmean ################################
def Any_Gmean(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return 1-sur_gmean

################################ WBCEGL ################################
def WBCEGL(y_true, y_pred):
    WBCEloss = WBCE(y_true, y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return (1-r)*WBCEloss+(r)*(1-sur_gmean)

################################ SPLGL ################################
def SPLGL(y_true, y_pred):
    SPL = splitter(y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return (1-w)*SPL+(w)*(1-sur_gmean)

# =================================== BAccu (Type0, 1, 2, 3) =================================== #
################################ Pure_BAccu ################################
def Pure_BAccu(y_true, y_pred):
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return 1-baccu

################################ Any_BAccu ################################
def Any_BAccu(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return 1-baccu

################################ WBCEBL ################################
def WBCEBL(y_true, y_pred):
    WBCEloss = WBCE(y_true, y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return (1-r)*WBCEloss+(r)*(1-baccu)

################################ SPLBL ################################
def SPLBL(y_true, y_pred):
    SPL = splitter(y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return (1-w)*SPL+(w)*(1-baccu)